In [ ]:
import datetime
def cell_format(v):
    return str(v) if isinstance(v, (int)) else str(int(v)) if isinstance(v,float) else str(v) if isinstance(v, datetime.date) else v.decode('utf-8') if isinstance(v, bytes) else '' if v is None else v.strip()
import sys
sys.path.append('/odm_modules')
from common_func import odm_conn
from common_func import odm_dict
import pandas as pd
from common_func import cloudant_conn as cc
import logging                                                                  
import os  
#sys.path.append('/app')
#from BOX import box_oauth as box 
pd.options.display.max_rows = 1000
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 0
odm_dict.refresh()

In [ ]:
df_tid = odm_dict.odm_tid_df_from_csv()
df_col = odm_dict.odm_col_df_from_csv()
df_col = df_col.loc[df_col.NAME !=''] # remove those line which could not be found in systable
df_tid = df_tid.loc[df_tid.NAME !='']
tid_dict = df_tid.set_index('CTID')['CTABNAME'].to_dict()
tid_desc_dict = df_tid.set_index('CTID')['TTID'].to_dict()
df_col['tid_col'] = df_col.CTID + '.' + df_col.CCOLNAME
data_class_dict = df_col.set_index('tid_col').CCLASS.to_dict()
col_dict = df_col.set_index('tid_col').TCOLNAME.to_dict()

## Get the dump for a given OMEGA feeder code

In [ ]:
fdr = 'HI7'

In [ ]:
tbs = df_tid.loc[df_tid.CTID.str.startswith('G0'), ['CTID', 'CTABNAME']].set_index('CTID')['CTABNAME'].to_dict()

In [ ]:
def get_result(tid):
    tbname = tbs[tid]
    sql = f'''select {tid}.* from ODMPRD.{tbname} {tid}, odmprd.odmt_ome_employee A 
    WHERE 
    {tid}.CCOUNTRY = A.CCOUNTRY 
    AND {tid}.CCOUNTRQ = A.CCOUNTRQ
    AND {tid}.RSERNUM = A.RSERNUM
    AND A.CFDRSRC = '{fdr}'  '''
    print(sql)
    with odm_conn.odm_adhoc('prod') as odmprd_adhoc:
        result = odmprd_adhoc(sql)
    df = pd.DataFrame(result).applymap(cell_format)
    df.to_excel(f'/result/result_{tid}_{fdr}.xlsx', index = False)
    return df

In [ ]:
dfs = {tid: get_result(tid) for tid in tbs.keys()}